In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd

data_csv = pd.read_csv('/kaggle/input/fsc22-dataset/Metadata-20220916T202011Z-001/Metadata/Metadata V1.0 FSC22.csv')




In [2]:
data_csv


,Source File Name,Dataset File Name,Class ID,Class Name
0,17548__A.wav,1_10101.wav,1,Fire
1,17548_B.wav,1_10102.wav,1,Fire
2,17548_C.wav,1_10103.wav,1,Fire
3,17548_D.wav,1_10104.wav,1,Fire
4,17548_E.wav,1_10105.wav,1,Fire
...,...,...,...,...
2020,164882__timsc__squirrel-call.wav,27_12771.wav,27,Squirrel
2021,162648__cognito-perceptu__park-bench-atmospher...,27_12772.wav,27,Squirrel
2022,122260__echobones__angry-squirrel-long.wav,27_12773.wav,27,Squirrel
2023,82828__noisecollector__angrysquirrel-creepingt...,27_12774.wav,27,Squirrel


We have 2025 data sets as our audio data sets.

to Augment the data set and widen it we pitch 2 steps up and down in each audio file

In [3]:
import librosa
import soundfile as sf
from tqdm import tqdm
import os

input_dir = "/kaggle/input/fsc22-dataset/Audio Wise V1.0-20220916T202003Z-001/Audio Wise V1.0"

output_dir = "/kaggle/working/FSC22_augmented"
os.makedirs(output_dir, exist_ok = True)


PITCH_STEPS = [2,-2]

for file in tqdm(os.listdir(input_dir)):
    if file.endswith(".wav"):
        file_path = os.path.join(input_dir, file)

        y, sr = librosa.load(file_path, sr=None) #loading the audio

        sf.write(os.path.join(output_dir, file),y,sr)

        for step in PITCH_STEPS:
            y_shifted = librosa.effects.pitch_shift(y,sr= sr, n_steps=step)
            base, ext = os.path.splitext(file)
            new_filename=f"{base}_pitch{step}{ext}"
            sf.write(os.path.join(output_dir, new_filename), y_shifted, sr)

print("Augmentation Completed!")




100%|██████████| 2025/2025 [10:20<00:00,  3.27it/s]

Augmentation Completed!


In [6]:
filename_to_label= {}

for i, row in data_csv.iterrows():
    filename = row["Dataset File Name"]
    class_id = row["Class ID"]
    filename_to_label[filename] = class_id

def get_orginal_filename(filename):
    base, ext = os.path.splitext(filename)
    if "_pitch" in base:
        base = base.split("_pitch")[0]
    return base + ext
    

Since we have augmented the data set now we are trying to do feature extraction.

for ausio signal processing, we use MFCC and melspectrogram methods using librosa library

In [19]:
import numpy as np
auto_dir = "/kaggle/working/FSC22_augmented"

#Feature Extraction Parameters

SR = 22050 #sampling rate
N_FFT = 2048  #number of samples per FFT window
HOP_LENGTH = 512 #
N_MELS = 128 #number of mel frequency bands. 128 is ide
N_MFCC = 13 #number of MFCC coefficients

def extract_mel_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=SR) # Y is the 1D waveform array
    mel_spec = librosa.feature.melspectrogram( y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)
    mel_specs_db = librosa.power_to_db(mel_spec, ref=np.max)

    return mel_specs_db

def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=SR)
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=N_MFCC,n_fft=N_FFT,hop_length=HOP_LENGTH)

    #MFCC algorthm works as.. STFT ->MEL filterbank(triangular) -> log -> DCT
    return mfcc



#Looping 


mel_features = []
mfcc_features = []
file_names = []

for file in tqdm(os.listdir(auto_dir)):
    if file.endswith(".wav"):

        path = os.path.join(auto_dir, file)

        
        mel =extract_mel_spectrogram(path)
        mfcc = extract_mfcc(path)

        mel_features.append(mel)
        mfcc_features.append(mfcc)
        file_names.append(file)

mel_features = np.array(mel_features, dtype = object)
mfcc_features = np.array(mfcc_features, dtype = object)


        

100%|██████████| 6075/6075 [03:48<00:00, 26.62it/s]


In [39]:
import os
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import librosa

x = []
y = []

#preparing 1D feature 

for mfcc, fname in zip(mfcc_features, file_names):
    orginal_fname = get_orginal_filename(fname)
    label = filename_to_label[orginal_fname]

    mfcc_mean = np.mean(mfcc,axis =1)
    x.append(mfcc_mean)
    y.append(label)
    




In [49]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split( x, y, test_size=0.2, random_state=42, stratify = y)

for i in range(len(y_train)):
    y_train[i] = y_train[i]-1

for j in range(len(y_val)):
    y_val[j] = y_val[j] - 1


import xgboost as xgb

dtrain =xgb.DMatrix( X_train, label = y_train)
dval = xgb.DMatrix(X_val, label=y_val)

params = {
    "objective": "multi:softmax",
    "num_class" : 27,
    "eval_metric":"merror",
    "subsample":1,
    "min_child_weight":1,
    "max_depth":6,
    "learning_rate":0.3,
    "eval_metric" : "mlogloss"
}

num_rounds = 100

model = xgb.train(
    params,
    dtrain,
    num_boost_round = num_rounds,
    evals=[(dval, "validation")])

preds = model.predict(dval)
accuracy = (preds==y_val).mean()
print("validation _accuracy:", accuracy)

[0]	validation-mlogloss:2.60659
[1]	validation-mlogloss:2.34088
[2]	validation-mlogloss:2.16948
[3]	validation-mlogloss:2.03387
[4]	validation-mlogloss:1.93115
[5]	validation-mlogloss:1.83982
[6]	validation-mlogloss:1.76980
[7]	validation-mlogloss:1.70440
[8]	validation-mlogloss:1.65077
[9]	validation-mlogloss:1.60982
[10]	validation-mlogloss:1.57009
[11]	validation-mlogloss:1.53826
[12]	validation-mlogloss:1.50220
[13]	validation-mlogloss:1.47120
[14]	validation-mlogloss:1.44174
[15]	validation-mlogloss:1.41127
[16]	validation-mlogloss:1.38332
[17]	validation-mlogloss:1.36354
[18]	validation-mlogloss:1.34421
[19]	validation-mlogloss:1.32537
[20]	validation-mlogloss:1.30565
[21]	validation-mlogloss:1.28389
[22]	validation-mlogloss:1.27097
[23]	validation-mlogloss:1.25776
[24]	validation-mlogloss:1.24447
[25]	validation-mlogloss:1.23148
[26]	validation-mlogloss:1.21740
[27]	validation-mlogloss:1.20877
[28]	validation-mlogloss:1.20194
[29]	validation-mlogloss:1.18993
[30]	validation-mlog